In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

c:\users\dell\anaconda3\envs\tf\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
data= pd.read_csv('./data1/training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')

In [4]:
data.head()

essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   rater1_domain1  rater2_domain1  rater3_domain1  domain1_score  \
0               4               4             NaN              8   
1               5               4             NaN              9   
2               4               3             NaN              7   
3               5               5             NaN             10   
4               4               4             NaN              8   

   rater1_domain2  rater2_domain2  domain2_score  ...  rater2_trait3  \
0             NaN             NaN            NaN  ...            NaN   
1             NaN             NaN            NaN  ...            NaN   
2             NaN             NaN            NaN  ...            NaN   
3             NaN             NaN            NaN  ...            NaN   
4             NaN             NaN            NaN  ...            NaN   

   rater2_trait4  rater2_trait5  rater2_trait6  rater3_trait1  rater3_trait2  \
0            NaN            NaN            NaN            NaN            NaN   
1            NaN            NaN            NaN            NaN            NaN   
2            NaN            NaN            NaN            NaN            NaN   
3            NaN            NaN            NaN            NaN            NaN   
4            NaN            NaN            NaN            NaN            NaN   

   rater3_trait3  rater3_trait4  rater3_trait5  rater3_trait6  
0            NaN            NaN            NaN            NaN  
1            NaN            NaN            NaN            NaN  
2            NaN            NaN            NaN            NaN  
3            NaN            NaN            NaN            NaN  
4            NaN            NaN            NaN            NaN  

[5 rows x 28 columns]

In [5]:
data.isnull().any()

essay_id          False
essay_set         False
essay             False
rater1_domain1    False
rater2_domain1    False
rater3_domain1     True
domain1_score     False
rater1_domain2     True
rater2_domain2     True
domain2_score      True
rater1_trait1      True
rater1_trait2      True
rater1_trait3      True
rater1_trait4      True
rater1_trait5      True
rater1_trait6      True
rater2_trait1      True
rater2_trait2      True
rater2_trait3      True
rater2_trait4      True
rater2_trait5      True
rater2_trait6      True
rater3_trait1      True
rater3_trait2      True
rater3_trait3      True
rater3_trait4      True
rater3_trait5      True
rater3_trait6      True
dtype: bool

In [6]:
data= data.dropna(axis=1)
data= data.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [7]:
data.head()

essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   domain1_score  
0              8  
1              9  
2              7  
3             10  
4              8

In [8]:
x=data.iloc[:,0:3]
y=data.iloc[:,3]

In [9]:
x

essay_id  essay_set                                              essay
0             1          1  Dear local newspaper, I think effects computer...
1             2          1  Dear @CAPS1 @CAPS2, I believe that using compu...
2             3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...
3             4          1  Dear Local Newspaper, @CAPS1 I have found that...
4             5          1  Dear @LOCATION1, I know having computers has a...
...         ...        ...                                                ...
12971     21626          8   In most stories mothers and daughters are eit...
12972     21628          8   I never understood the meaning laughter is th...
12973     21629          8  When you laugh, is @CAPS5 out of habit, or is ...
12974     21630          8                                 Trippin' on fen...
12975     21633          8   Many people believe that laughter can improve...

[12976 rows x 3 columns]

In [10]:
y

0         8
1         9
2         7
3        10
4         8
         ..
12971    35
12972    32
12973    40
12974    40
12975    40
Name: domain1_score, Length: 12976, dtype: int64

In [11]:
min_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
max_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

# preprocessing

In [12]:
#removing the extra characters other than alphabets and stopwords and tokenizing the words
def essay_to_wordlist(essay_v):
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    return (words)

#Tokenize the senstences and call essay_to_wordlist() for word tokenization.
def essay_to_sentences(essay_v):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence))
    return sentences


In [13]:
#Feature vector is made from the words list of an essay.
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model.wv[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

#Word vectors are generated for Word2Vec model
def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [14]:
#the dataset is split to training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 0)

In [15]:
x_train.shape

(10380, 3)

In [16]:
x_test.shape

(2596, 3)

In [17]:
train_essays = x_train['essay']
test_essays = x_test['essay']

In [18]:
sentences = []
# Obtaining all sentences from the training essays.
for essay in train_essays:
    sentences += essay_to_sentences(essay)

In [19]:
sentences

[['essay', 'author', 'talking', 'journey'],
 ['shows',
  'give',
  'ends',
  'almost',
  'way',
  'began',
  'shows',
  'people',
  'stay',
  'attached',
  'past',
  'reasons',
  'believe',
  'author',
  'concludes',
  'story',
  'paragraph'],
 ['shows',
  'people',
  'give',
  'even',
  'though',
  'failed',
  'test',
  'try',
  'later',
  'pass'],
 ['also',
  'story',
  'ends',
  'way',
  'begins',
  'taking',
  'test',
  'getting',
  'ready',
  'good',
  'way',
  'writing'],
 ['one',
  'last',
  'thing',
  'shows',
  'people',
  'become',
  'attached',
  'past',
  'look',
  'one',
  'thing',
  'reminds',
  'month',
  'fall',
  'apart',
  'takes',
  'test',
  'see',
  'look',
  'familiar',
  'flowers',
  'without',
  'falling',
  'apart'],
 ['story',
  'winter',
  'hibiscus',
  'minfong',
  'ho',
  'think',
  'ending',
  'paragraph',
  'show',
  'saeng',
  'given'],
 ['says', 'come', 'back', 'take', 'test', 'talking', 'hibiscus', 'geese'],
 ['things', 'remind', 'home'],
 ['reminders'

In [20]:
#build the vectorizer with maximum featuroes of 300
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3
model = Word2Vec(sentences, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, 
                 sample = downsampling)             

In [21]:
#save the vectorizer in .bin file
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

In [22]:
#get the training vectors
clean_train_essays = []
for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
 
#get the testing vectors
clean_test_essays = []
for essay_v in test_essays:
    clean_test_essays.append(essay_to_wordlist( essay_v))
testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
#convert the vectors to numpy array
trainDataVecs = np.array(trainDataVecs)
testDataVecs = np.array(testDataVecs)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

# model building

In [39]:
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [41]:
from tensorflow.keras import models

In [44]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from tensorflow.keras.models import Sequential, load_model, model_from_config
import tensorflow.keras.backend as k

In [45]:
model = Sequential()
model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
model.add(LSTM(64, recurrent_dropout=0.4))
model.add(Dropout(0.5))
model.add(Dense(1, activation='relu'))

In [46]:
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])

In [47]:
model.fit(trainDataVecs, y_train, batch_size=64, epochs=11)

Epoch 1/11
163/163 [==============================] - 14s 29ms/step - loss: 53.1677 - mae: 3.5755
Epoch 2/11
163/163 [==============================] - 5s 28ms/step - loss: 28.5826 - mae: 2.4508
Epoch 3/11
163/163 [==============================] - 5s 29ms/step - loss: 19.0827 - mae: 2.1192
Epoch 4/11
163/163 [==============================] - 5s 28ms/step - loss: 13.3637 - mae: 1.8895
Epoch 5/11
163/163 [==============================] - 5s 28ms/step - loss: 10.5405 - mae: 1.7322
Epoch 6/11
163/163 [==============================] - 5s 28ms/step - loss: 8.8857 - mae: 1.6312
Epoch 7/11
163/163 [==============================] - 5s 28ms/step - loss: 8.4778 - mae: 1.5912
Epoch 8/11
163/163 [==============================] - 5s 28ms/step - loss: 8.5420 - mae: 1.5833
Epoch 9/11
163/163 [==============================] - 5s 29ms/step - loss: 8.3529 - mae: 1.5438
Epoch 10/11
163/163 [==============================] - 5s 28ms/step - loss: 8.0105 - mae: 1.5200
Epoch 11/11
163/163 [============

In [52]:
testDataVecs.shape

(2596, 1, 300)

In [53]:
y_pred = model.predict(testDataVecs)

In [54]:
y_pred

array([[13.971834 ],
       [19.171095 ],
       [ 2.5213337],
       ...,
       [ 1.8073874],
       [ 2.44984  ],
       [ 3.1793518]], dtype=float32)

In [55]:
model.save('final_lstm.h5')

In [56]:
from sklearn.metrics import r2_score
accuracy = r2_score(y_test,y_pred)
accuracy

0.930282438766901

# model testing

In [57]:
import gensim.models.keyedvectors as word2vec

In [58]:
testsen='''Dear local newspaper, I\'ve heard that not many people think computers benefit society. I disagree with that. Computers benefit society by teaching hand-eye coordination, allowing people to learn about foregin places, and allow people to communicate with others online. Some people were basically born with hand-eye coordination, but most people weren\'t, like me, for example. But, thanks to computers, my hand-eye coordination has gone up! "On average, computers can increase a persons hand-eye coordination by up to @PERCENT1 says @PERSON2. Think of it, just by learning to tye, your hand-eye coordination can go up @PERCENT1.! That\'s amazing! Having good hand-eye coordination will get you far in life, and helps with small, everyday things such as playing an instrument, or doing household chores. It also helps in sports. You need good hand-eye coordination to throw, catch, and aim. If more people spend more time on the computer, then the population\'s hand-eye coordination would increase, and who wouldn\'t want thats. Have you ever heard of @LOCATION1? It is a country in @LOCATION2 that no one really knows much about. By looking it up on the computer, you can become an expert on the country! The computer gives us access to a lot of knowledge, and will make us more of people around the world. This can improve our travel. Instead of going to the same place every vacation, you can go to a place you saw online that looks just as exciting. Also, by learning about other countries and the people in them, we gain respect for them. We @MONTH1 also learn about their traditions, and culture. "The internet gives us access to the world," says @PERSON1, "we can learn so many new things that will make us more of people, countries, and the environment." @CAPS1 is right, we can also be aware of the environment. Since global warming is a problem, we can learn how o preserve foreign places by learning about them on the internet. With a click of a mouse you can virtually have the whose world at your fingertips! In a recent study @PERCENT2 of people said they don\'t have time to meet with old friends. The internet, however has made it quick and easy to talk to people online. Websites like @CAPS2, @CAPS3, and @CAPS4 can keep your social life in tact, and can even reunite yu with old friends. For example, @CAPS2 suggest family. You can keep in touch wih many people trough @CAPS2! Also, you can make new friends. Most teens say they are always willing to meet new people, and the internet is a great way to do so! Sites like @CAPS7, and @CAPS8 can can help you find friends by connecting you in a chat room to a random person, just to talk. I once talked with someone on @CAPS7 for over an hour! We found that we both have @CAPS4 accounts and keep in touch there! This also helps you gain confidence because you @MONTH1 have the courage to say something over the computer that you would\'t say real life. The computer can really amp up your social life. As you can see. computers don\'t just "your barain," they imprdove hand-eye coordination, help you learn about far away places, and improve your social life by allowing you to talk to others. Computers benefit the society more than anymore will ever know!'''

In [59]:
testsen

'Dear local newspaper, I\'ve heard that not many people think computers benefit society. I disagree with that. Computers benefit society by teaching hand-eye coordination, allowing people to learn about foregin places, and allow people to communicate with others online. Some people were basically born with hand-eye coordination, but most people weren\'t, like me, for example. But, thanks to computers, my hand-eye coordination has gone up! "On average, computers can increase a persons hand-eye coordination by up to @PERCENT1 says @PERSON2. Think of it, just by learning to tye, your hand-eye coordination can go up @PERCENT1.! That\'s amazing! Having good hand-eye coordination will get you far in life, and helps with small, everyday things such as playing an instrument, or doing household chores. It also helps in sports. You need good hand-eye coordination to throw, catch, and aim. If more people spend more time on the computer, then the population\'s hand-eye coordination would increase,

In [60]:
model = word2vec.KeyedVectors.load_word2vec_format('word2vecmodel.bin', binary=True)
index2word_set = set(model.index_to_key)

In [61]:
testsen2 = re.sub("[^a-zA-Z]", " ", testsen)
testsen2 = essay_v.lower()
featureVec = np.zeros((300,),dtype="float32")
for word in testsen2:
        if word in index2word_set:
            featureVec = np.add(featureVec,model[word])

In [62]:
featureVec.shape

(300,)

In [63]:
avc=featureVec.reshape(1,1,300)

In [65]:
from tensorflow.keras.models  import load_model
model1 = load_model("final_lstm.h5")

In [66]:
y_pred = model1.predict(avc)

In [67]:
y_pred

array([[5.793888]], dtype=float32)